In [ ]:
"""
In this notebook there are four main cells

Cell 1 creates a QUBO Adjacency to represent the ambulance problem, it involves constraints and minimising distance driven.
Cell 2 Defines the methods used in Cell 3, and converts QUBO Adjacency to the dwave type bqm that can be sampled 
Cell 3 Finds the minimum energy for a given starting postion of two ambulances, using a choice of a) dimod.ExactSolver() b)neal.SimulatedAnnealingSampler()   or c) DWaveSampler() the live QPU not a simulator or d) TabuSampler()
cell 4 Classically iterates over all possible starting position of two ambulances and selects the lowest energy of them all and which destination are allocated to each of the two ambulances.

In [11]:
#TWO AMBULANCE DISTANCE MINIMISATION - 

import numpy as np 
#RC utility functions    
from pyaqc.RCModules.PlatformConversion import print_QUBOdetails, CreateTwoAmbulanceAdjacencyV1,CreateTwoAmbulanceAdjacencyV2
#################################### Create Adjacency defined as a qubo ####################################
n_destinations = 3*2
gridWidth = 3# n_destinations
Width  = gridWidth
Height = n_destinations//gridWidth

result = CreateTwoAmbulanceAdjacencyV2(gridWidth,n_destinations, Adddistance = 1, ConstraintMultiplier=5, use_XYMixer_constraints=0)

Adjacency = result['qubo']
AdjacencyHybrid = result['quboHybrid']
n_qubits = result['n_qubits']
ConstraintMultiplier = result['ConstraintMultiplier']
max_distance = result['max_distance']

print(ConstraintMultiplier/max_distance ,' = ConstraintMultiplier/max_distance')
if 1:
    filename = 'Twoambulances.txt'
    
    print_QUBOdetails(Adjacency,n_qubits,filename)

1.0  = ConstraintMultiplier/max_distance
ADJACENCY MATRIX Twoambulances.txt  

Qubit  q0   q1   q2   q3   q4   q5   q6   q7   q8   q9  q10  q11  q12  q13  q14  q15  q16  q17  q18  q19  q20  q21  q22  q23 

  q0   -5                            10                                 1    4    1    2    5  

  q1        -5                            10                       1         1    2    1    2  

  q2             -5                            10                  4    1         5    2    1  

  q3                  -5                            10             1    2    5         1    4  

  q4                       -5                            10        2    1    2    1         1  

  q5                            -5                            10   5    2    1    4    1  

  q6                                 -5                                                               1    4    1    2    5  

  q7                                      -5                                              

In [12]:
#Myqubo = dimod.BinaryQuadraticModel.from_ising(linear, quadratic,n_qubits)      #the length of linear defines the number of qubits in the qubo
import matplotlib.pyplot as plt
import dimod
samplerExact = dimod.ExactSolver()
def lowest_energy_num_occ(response, atol:'tolerance'=0):
    """
    Returns (type:float) the number of samples in the 'response' (type:sample_set) with energies < lowest_energy + atol
    params:  'response' (type:sample_set)
              'atol' (type:float)
          
    """
    num_occ = 0
    for data in  response.data(['num_occurrences','energy']):
            if data.energy < response.first[1]+ atol :
                num_occ += data.num_occurrences
#                print(data.energy)
            else:
#                print(data.energy)
                break
    return num_occ
        
def create_hybrid_from(Adjacency, Nlocs,A0_fix ,A1_fix ):
    """
    returns (Type:dict) A simplified adjacency table, (a subproblem) that uses a fixed starting position for each ambulance
    """
    for L in range(0 , Nlocs):
        # remove AdjacencyHybrid distance references
        if (L, 2*Nlocs) in AdjacencyHybrid:
            del AdjacencyHybrid[(L, 2*Nlocs)] 
        if (L + Nlocs, 2*Nlocs +1) in  AdjacencyHybrid:
            del AdjacencyHybrid[(L + Nlocs, 2*Nlocs +1)] 
        
        # add first ambulance distances from fixed start to each destinations (if non-zero)
        A0_fix_to_des_key = (L, A0_fix + 2 * Nlocs) 
        if (A0_fix_to_des_key) in  Adjacency:
            AdjacencyHybrid[(L, 2*Nlocs)] = Adjacency [A0_fix_to_des_key]                 
        # then second ambulance
        A1_fix_to_des_key = (L + Nlocs, A1_fix + 3 * Nlocs )
        if (A1_fix_to_des_key) in  Adjacency:
            AdjacencyHybrid[(L + Nlocs, 2*Nlocs +1)] = Adjacency [A1_fix_to_des_key]       
    return AdjacencyHybrid
from dwave.cloud import Client
#client = Client.from_config(token='DEV-d58203bae0f10bbf7a7bec839b170624758a9d5d')       #irgambler actual token
client = Client.from_config(token='DEV-f94593d3e9df93c9227b0caeeae3734e793b8feb')       #rcgquantum@gmail.com
client.get_solvers()

[StructuredSolver(id='DW_2000Q_6'), StructuredSolver(id='Advantage_system1.1')]

In [13]:
#This cell creates a Ising model, from a qubo defined adjacency, where the starting positions of the two  ambulances have been specified by the user
# eg Amulance 'zero' starts location number 4 so set A0_fix=4
#Then this cell finds the lowest energy waveform using one of three different minimisation approaches    
#Cell 3 Finds the minimum energy for a given starting postion of two ambulances, using a choice of a)dimod.ExactSolver() b)neal.SimulatedAnnealingSampler()   or c) DWaveSampler()
UseHybrid = 0
sampler_choice = ['samplerExact','simAnneal','DWave_pegasus','DWave_chimera', 'Tabu']
Method = sampler_choice[2]
Num_readsMy = 1     # SolverFailureError: The parameter num_reads must be within [1, 10000] for the qpu
my_prefactor = 1.25 #1.414
################# Main minimisation of the subproblem #################
from datetime import datetime
from pyaqc.RCModules.TwoAmbulanceAnalysis import *
tstart = datetime.now()
if Height==2 and Width==2:
    A0_fix = 1
    A1_fix = 2
if Height==3 and Width==3:
    A0_fix = 1
    A1_fix = 7
if Height==5 and Width==5: #On inspection 7, 17 is the optimized distance start positions for 5*5
    A0_fix = 7 #22                          # a 10*5 grid has a more obvious solution by inspection 22,27
    A1_fix=  17 # 27#  16,32  40  
if Height==7 and Width==7:      #I need to check this is the optimum position of the ambulances
    A0_fix = 16
    A1_fix = 32                   
ConstraintMet = 1
Nlocs = Height*Width
MinE = 0

if UseHybrid:
    AdjacencyHybrid = create_hybrid_from(Adjacency, Nlocs,A0_fix ,A1_fix )
    Myqubo = dimod.BinaryQuadraticModel.from_qubo(AdjacencyHybrid)       #use quboHybrid with given A0_fix and A1_fix
else:
    Myqubo = dimod.BinaryQuadraticModel.from_qubo(Adjacency)             #use Adjacency with all starting positions

############ Use one of three possible samplers
    # a)samplerExact(calculated every possible combination of ambulance start positions)
    # b)SimulatedAnnealingSampler()  (classically imitates a Dwave annealer)
    # c)sampler = EmbeddingComposite(DWaveSampler()) , the real online Dwave annealer

if Method == sampler_choice[0]:       # 'samplerExact'
    response = samplerExact.sample(Myqubo) 
    Num_readsMy = 1
    energy = response.first[1]
    
if Method == sampler_choice[1]: # Method == 'simAnneal'
    import neal
    solver = neal.SimulatedAnnealingSampler()  
    response = solver.sample(Myqubo, num_reads=Num_readsMy)
    plt.plot(response.data_vectors['energy'])



from dwave.system.samplers import DWaveSampler
from dwave.system.composites  import EmbeddingComposite
if (Method == sampler_choice[2] or  Method == sampler_choice[3]) and 1:
    from dwave.embedding.chain_strength import uniform_torque_compensation      #prefactor * rms * math.sqrt(avg_degree)
    if 1:
        
        chain_strength_value = uniform_torque_compensation(Myqubo.spin, embedding=None, prefactor=my_prefactor)
    else:
        chain_strength_value = 1
    
    anneal_scheduleMy = [[0,0],  [20, 1]] #[10, 0.4], [40, 0.6],
if Method == sampler_choice[2]: # DWAVE online annealing Method = 'DWave_pegasus'
    sampler = EmbeddingComposite(DWaveSampler(solver={'topology__type': 'pegasus'}))        #'chimera ', 'pegasus'
    response = sampler.sample(Myqubo, num_reads=Num_readsMy,chain_strength=chain_strength_value,return_embedding=True)
if Method == sampler_choice[3]: # DWAVE online annealing Method = 'DWave_chimera' with anneal_scheduleMy
    sampler = EmbeddingComposite(DWaveSampler(solver={'topology__type': 'chimera'}))        #'chimera ', 'pegasus'
    response = sampler.sample(Myqubo, num_reads=Num_readsMy,return_embedding=True,anneal_schedule=anneal_scheduleMy,chain_strength=chain_strength_value)#, anneal_schedule=anneal_scheduleMy)

from tabu import TabuSampler
if Method == sampler_choice[4]:     #'Tabu'
    response = TabuSampler().sample(Myqubo, num_reads=Num_readsMy)
TimeTaken = (datetime.now() - tstart).seconds
print(TimeTaken, '= Time taken (secs) by', Method)    

# Find Start positions in the solution
psi_opt = list( response.first[0].values())
if not UseHybrid and 1:
    A0_sol  =   Grid_StartPositionsAO(psi_opt,  Height,Width)
    A1_sol  =   Grid_StartPositionsA1(psi_opt,  Height,Width)
else:
    A0_sol = A0_fix
    A1_sol  = A1_fix
Print_Destinations(response, Height,Width,Method, A0_sol, A1_sol,show_start_pos=1)
if A0_sol>=0 and  A1_sol>=0:
    Distance = SolutionDistance(psi_opt,A0_sol,A1_sol,Adjacency,Nlocs)
    print('Distance of solution = ', Distance, end='')
else:
    print('Start positions not found')
    Distance = -1
print( '\tWith A0_sol =',A0_sol,'and A1_sol =',A1_sol)
print( ' ConstraintMultiplier/max_distance = ', ConstraintMultiplier/max_distance  )
if 1:
    repeats = lowest_energy_num_occ(response,0.01)
    print(repeats, " = number of occurances of the lowest energy found. Using num_shots =  {0:4} ".format( Num_readsMy )  )
    print('%3.2f'%(repeats*100/Num_readsMy), '= % Probability that this energy level was found by one anneal, ie num_shot=1')
    print(len(response.data_vectors['energy']), 'len (List of  energies UNORDERED)')

    print(              '{:15} {:15} {:15}{:15}{:15}{:15}{:15}{:15}{:15}{:15}'.format('Grid','Energy', 'Distance', 'Errs','Constraintmet','Method', 'Lagrange K', 'num_reads ', 'time', 'my_prefactor'),end=''  )
    print("\n|{:<2} {:<1}{:<1}|{:>14}|{:15}|>0 |{:>15}|{:>15s}| {:15}|{:15}|{:15}|".format(Width,'* ',Height,response.first[1],Distance,ConstraintMet, Method,ConstraintMultiplier/max_distance,Num_readsMy,TimeTaken) ,end='')
    print(repeats,'(', '%3.2f'%(repeats*100/Num_readsMy),'%)|', '%3.2f'%my_prefactor,'|')
if 0:
    print(response.info['embedding_context']['chain_strength']  )

2 = Time taken (secs) by DWave_pegasus
StartPositionsA0
[0, 1, 0]
[0, 0, 0]
			 Start Position A0 Constraints met
StartPositionsA1
[1, 0, 0]
[1, 0, 0]
 			ERROR Start Position A1 Constraints not met
-29.0  = Lowest energy found by DWave_pegasus  in a grid 3 (w) by 2 (h)
Destination Constraints were Met.
 Map of destinations of each ambulance, 1 for A0, 0 for A1. Where A0 starts add 10, A1 add 20
[1, 11, 1]
[20, 1, 1]
Distance of solution =  5	With A0_sol = 1 and A1_sol = 3
 ConstraintMultiplier/max_distance =  1.0
1  = number of occurances of the lowest energy found. Using num_shots =     1 
100.00 = % Probability that this energy level was found by one anneal, ie num_shot=1
1 len (List of  energies UNORDERED)
Grid            Energy          Distance       Errs           Constraintmet  Method         Lagrange K     num_reads      time           my_prefactor   
|3  * 2|         -29.0|              5|>0 |              1|  DWave_pegasus|             1.0|              1|              2|1 (

In [14]:
import dwave.inspector
dwave.inspector.show( response)

'http://127.0.0.1:18000/?problemId=31a39b18-b36c-43e6-9c3d-83c067c54363'

In [56]:
print(response.first[3])        #chain break fraction in that sample
print('UNPACK each sample from the sampleSet, in ORDER of (energy, or of evaluations)  into\n', 'Sample', '\t\t\t\t\t\tenergy','\tnum_occurrences', 'chain_break_fraction')
for n,datum in enumerate( response.data(fields=['sample', 'energy','num_occurrences','chain_break_fraction'], sorted_by='energy')):     #'energy''chain_break'
    if n<20:
        print(list(datum.sample.values() ), '\t', datum.energy ,'\t', datum.num_occurrences,'\t',datum.chain_break_fraction)

0.0
UNPACK each sample from the sampleSet, in ORDER of (energy, or of evaluations)  into
 Sample 						energy 	num_occurrences chain_break_fraction
[0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0] 	 -30.0 	 1 	 0.0


In [15]:
response.info['embedding_context'] ['embedding']

{0: (842,),
 6: (3363,),
 13: (3469, 962),
 14: (3394, 932, 3393),
 15: (3424,),
 16: (3334, 872),
 17: (977, 3454),
 19: (752, 3423),
 20: (737, 3303),
 21: (691, 692),
 22: (782, 781, 3438),
 23: (767, 3273),
 2: (917, 918),
 3: (887,),
 4: (827,),
 5: (857,),
 12: (3439,),
 1: (3409, 947),
 7: (3289, 3288),
 18: (722, 3333),
 8: (3319, 3318),
 9: (3348, 3349),
 10: (3408,),
 11: (3378,)}

In [8]:
# DWave_pegasus_embedding   1.96  = average chain length. 3  = max_len.
# 
# DWave_chimera_embedding   3.08  = average chain length. 5  = max_len. n* k(4,4) 4 internal (to the 4*4) edges,  and 2 external, length 4 degree 6
# 'target QPU' is real embedding, light grey edges are the chain dark grey are logical edges/couplings
DWave_pegasus_embedding = {0: (237, 5131),
 6: (5220, 162),
 13: (238, 5281),
 14: (252, 5190),
 15: (281, 282),
 16: (5221, 312, 313),
 17: (327, 5236, 328),
 19: (5280,),
 20: (118, 117),
 21: (5250, 208),
 22: (5475, 148, 147),
 23: (5265, 102),
 2: (357, 5146),
 3: (5311,),
 4: (5355, 163),
 5: (5340,),
 12: (296, 297),
 1: (192, 5175),
 7: (177, 5235),
 18: (5295,),
 8: (5145, 132),
 9: (5310, 87),
 10: (5325, 58),
 11: (5400, 193)}
# 3.08  = average chain length. 5  = max_len.
DWave_chimera_embedding = {0: (920, 792),
 6: (1048, 1176),
 13: (783, 799, 791, 785),
 14: (782, 798, 790, 787),
 15: (924, 916, 908, 921, 793),
 16: (780, 788, 796, 777),
 17: (781, 797, 789),
 19: (1045, 1053, 1037, 1040),
 20: (1177, 1049, 1173, 1181),
 21: (1038, 1054, 1046, 1043),
 22: (1052, 1036, 1044),
 23: (1039, 1055, 1047, 1041),
 2: (776, 904),
 3: (923, 795),
 4: (786, 914),
 5: (794, 922),
 12: (919, 911, 912, 784, 927),
 1: (779, 907),
 7: (1163, 1035, 1165),
 18: (1162, 1166, 1174, 1182, 1034),
 8: (1032, 1160),
 9: (1051, 1179),
 10: (1170, 1042),
 11: (1050, 1178)}
total = 0
n=0
max_len = 0
for key, val in DWave_pegasus_embedding.items():
    max_len = max(max_len,len(val))
    total += len(val)
    n += 1
print('%3.2f'%(total/n), ' = average chain length.', max_len, ' = max_len.')

1.96  = average chain length. 3  = max_len.


In [ ]:

"""
Record of actual problems submitted on line ie to the DWAVEQPU compared to the SimulatedAnnealingSampler

DWAVEQPU solutions:
#4*3 solution, this means W=4 and H=3 hence there are 12 possible locations to start from.
  0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1] = psi_opt first ambulance destinations 
 [1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0] = psi_opt 2nd ambulance destinations -7690.0 in 15 msecs
 
 9*9                    1858         yes        DWAVEQPU       sum_distance              20            16ms
 10*10      embedding found(10*10*2+2= 202)      more qubits than 7*7*4
 11*11      embedding found(11*11*2+2= 244)      more qubits than 7*7*4  so perhaps highly connected nature of start/distance is problem
A0_fix = 22 A1_fix = 27 I have found best Distance of solution for 11*11, by inspection, = 200 



A0_fix = 16 A1_fix = 32 12.35pm
9*9   -14520154.0       1526      yes           simAnneal   sum_distance                  1000
A0_fix = 22 A1_fix = 27 I have found best Distance of solution, by inspection, = 200 
10*5                    450       yes           simAnneal   sum_distance                  50000
10*5                    425       yes           simAnneal   sum_distance                  50000
10*5                    385       yes           simAnneal   max_distance *10              50000
10*5                    365       yes           simAnneal   max_distance *10              50000
"""



In [ ]:
Fixed starting positions ie UseHybrid = 1
Grid            Energy          Distance       Constraintmet  Method         Lagrange K     num_reads      time 

3  * 3        -871.0  (11.7%)            9                   1       DWAVEQPU            10.0           9000              5

In [5]:
# To reduce the qubits used, select the ambulance STARTING POSITIONS, (n(n-1))/2 of them, CLASSICALLY , calc each energy with dwave pick the lowest.
from dwave.system.samplers import DWaveSampler
from dwave.system.composites  import EmbeddingComposite
            
Nlocs = Height*Width
psi_opt =[]
MinE = 0
A0_sol = -1
A1_sol = -1
from datetime import datetime
tstart = datetime.now()
#Create a set of smaller subproblems each called 'AdjacencyHybrid' each derived from a fixed starting position of each ambulance A0_fix and A1_fix

for A0_fix in range(0,Nlocs):
    for A1_fix in range(A0_fix +1 ,Nlocs):
        AdjacencyHybrid = create_hybrid_from(Adjacency, Nlocs,A0_fix ,A1_fix ) 
        #2) Find the lowest energy of the subproblem
        Myqubo = dimod.BinaryQuadraticModel.from_qubo(AdjacencyHybrid)
        if 0: 
            response = samplerExact.sample(Myqubo) 
            Method = 'samplerExact'
        elif 1:
            Method = 'simAnneal'
            import neal
            solver = neal.SimulatedAnnealingSampler()  
            response = solver.sample(Myqubo, num_reads=Num_readsMy)
        elif 0: 
            # DWAVE online annealing Lesson 4
            sampler = EmbeddingComposite(DWaveSampler())                                    
            response = sampler.sample(Myqubo, num_reads=Num_readsMy)
            Method = 'DWAVEQPU'
        
        Distance = SolutionDistance(response.first[0],A0_fix,A1_fix,Adjacency,Nlocs)
        show_grid = 1
        if  not show_grid:
            ψ  = list( response.first[0].values() )
            print('',ψ[:Width * Height],              '= ψ  first ambulance destinations')
            A_start =[ int(not bool(n-A0_fix)) for n in range(Nlocs)]
            print(A_start , ' = start position of AO')
            print(ψ[Width * Height:2*Width * Height], '= ψ  2nd ambulance destinations', response.first[1], 'Distance = ',Distance,'\n')
            A_start =[ int(not bool(n-A1_fix)) for n in range(Nlocs)]
            print(A_start , ' = start position of A1')
        else:
            if 0:
                ψ  = Print_Destinations(response, Height,Width,Method)
            print( 'Distance = ',Distance)
            
        # print('Use both ambulance1;', datum.sample[Width * Height*2],'and ambulance2;',datum.sample[Width * Height*2+1])
        energy = response.first[1]              #This is the lowest energy found by the
        if MinE > energy:
            best_response = response
            ψ =     list( response.first[0].values() )        #a list of the waveform evaluated
            MinE = energy
            E = qubo_energy_value(ψ, AdjacencyHybrid)
            psi_opt = ψ
            A0_sol = A0_fix
            A1_sol  = A1_fix
            print(psi_opt, MinE,' A0pos =',A0_sol,' A1pos =',A1_sol, 'Distance = ',Distance )
            print((datetime.now() - tstart).seconds, '= seconds elapsed')   
#Print lowest energy from the set of all starting positions
Distance = SolutionDistance(best_response.first[0],A0_sol ,A1_sol,
print( '(Distance travelled)^2 by ambulances in solution to objective function = ',Distance)Adjacency,Nlocs)
ψ  = Print_Destinations(best_response, Height,Width,Method)

print('\nAfter ', (datetime.now() - tstart).seconds, 'seconds, Final psi:')

print('finished hybrid')

Distance =  18
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1] -862.0  A0pos = 0  A1pos = 1 Distance =  18
1 = seconds elapsed
Distance =  18
Distance =  18
Distance =  10
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] -870.0  A0pos = 0  A1pos = 4 Distance =  10
7 = seconds elapsed
Distance =  11
Distance =  18
Distance =  11
Distance =  14
Distance =  18
Distance =  13
Distance =  9
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1] -871.0  A0pos = 1  A1pos = 4 Distance =  9
18 = seconds elapsed
Distance =  13
Distance =  11
Distance =  9
Distance =  11
Distance =  11
Distance =  10
Distance =  18
Distance =  14
Distance =  11
Distance =  18
Distance =  9
Distance =  9
Distance =  18
Distance =  13
Distance =  11
Distance =  9
Distance =  10
Distance =  9
Distance =  10
Distance =  11
Distance =  13
Distance =  18
Distance =  18
Distance =  18
Distance =  18


TypeError: SolutionDistance() takes 5 positional arguments but 6 were given

In [10]:
print( '(Distance travelled)^2 by ambulances in solution to objective function = ',Distance)Adjacency,Nlocs)
ψ  = Print_Destinations(best_response, Height,Width,Method)


-871.0  = Lowest energy found by simAnneal  in a grid 3 (w) by 3 (h)
Destination Constraints were Met. 
 Map of destinations of each ambulance, 1 for A0, 0 for A1
[1, 1, 1]
[0, 0, 0]
[0, 0, 0]
(Distance travelled)^2 by ambulances in solution to objective function =  9


In [ ]:
"""         
Grid        features    qubits  2^n            Method      Energy solution         Time/hours               Comment
3*2         *4           24     2^24 = 16m      dimod       -2367=-2374+9           2mins
3*2         *4           24     2^24 = 16m      C++         -2367=-2374+9           15mins               not faster than dimod

3*3         *4          = 36    2^36 = 69bn     C++         -2367=-2374+9           69bn/20m = 3500mins
3*3         *2 +2       = 20    (9*8)2^20 = 16m dimod/hybrid -2367=-2374+9          4 mins

c++ and ipynb gives E = -2367 = -2374 + 9 by symmetry the solution is clear so I tested that the Ising was low 
"""



In [19]:
print('CLASSICAL calculation of energies using dimod and qubo_energy_value() which should equal each other')
print('Location grid is ',Width,'wide and ',Height,'high.' )
Myqubo = dimod.BinaryQuadraticModel.from_qubo(qubo)
#THIRD calculate ENERGY of each state
tstart = datetime.now()
################ (A) calculate energy of one input state, using if 1 ################
if 0:
    #Below are three ψ solutions for location grids of 2*2, or 3*2, or 3*3 
    ψ = [1, 1, 1, 0, 0, 0, 0, 0,  0,1, 1, 1,0 ,1, 0, 0, 1, 0]  #16 qubits 2*2*4 = 16 qubits
    #   des_a0              des_a1              START_A0            START_A1
    #   0                   6                   12                  18
    ψ = [1, 1, 1, 0,0, 0   , 0, 0, 0,1, 1, 1,   1,0, 0, 0, 0, 0,    0,0, 0, 0, 1, 0]  #2*2,
    
    ψ = [1, 1, 1,1,1,0,0,0,0   , 0,0,0,0,0,1,1,1,1,   0,1,0,0,0,0,0,0,0,    0,0,0,0,0,0,0,1,0] #grid 3*2*4

    #ψ =[1, 1, 1, 0, 0, 0, 0, 0, 0  , 0, 0, 0, 1, 1, 1, 1, 1, 1,   1, 1] hybrid 3*3 gives E = -2367 takes 6 mins(3.5min not in debug mode or .ipynb) with energy = response.first[1]
    #...but 2.3 hrs with "enumerate" cycling through every energy level was costly
    E = qubo_energy_value(ψ, qubo)
    
    print('\t\t\t\t',ψ[:Width * Height],'= ψ first ambulance destinations','\n','\t\t\t\t',ψ[Width * Height:2*Width * Height],'= ψ 2nd ambulance destinations \t')
    print(ψ[2*Width * Height:3*Width * Height],'= ψ first ambulance Start position',ψ[3*Width * Height:4*Width * Height],'= ψ 2nd ambulance Start position\t', '%3.2f' %E,'= classical Energy')
else:
    ################ or (B) calculate energy of all input state, using if 0 ################
    response = samplerExact.sample(Myqubo) 
    stop =0 
    max_sol = 0
    MinE = -10000

    for n,datum in enumerate( response.data(['sample', 'energy'])):     
        if  n < 30 or 0:                                 #display lowest energy results
            for i, elem in enumerate( datum.sample):
                ψ[i] = (datum.sample[elem]) 
            #Check if Dwave energy calculation is the same as my calculation qubo_energy_value()
            E = qubo_energy_value(ψ, qubo)
    # print ERROR if the constraints have not been met. The target is for just one start postion per Ambulance, every location to be designated just one ambulance         
            if max(ψ[:Width * Height]+ ψ[Width * Height:2*Width * Height]) >1:
                print('ERROR',ψ[:Width * Height],'= ψ first ambulance destinations','\n',ψ[Width * Height:2*Width * Height],'= ψ 2nd ambulance destinations \t')
            if sum(ψ[3*Width * Height:4*Width * Height]) !=1 and sum(ψ[2*Width * Height:3*Width * Height]) !=1 and stop <20 :
                stop +=1
                print('ERROR',ψ[2*Width * Height:3*Width * Height],'= ψ first ambulance Start position',ψ[3*Width * Height:4*Width * Height],'= ψ 2nd ambulance Start position\t', '%3.2f' %E,'= classical Energy', '%3.2f' %datum.energy,'= DIMOD calc of energy n=',n)
    #print position of Ambulance start positions and destinations. 
            if MinE < datum.energy or  max_sol <5:
                max_sol += 1
                MinE = datum.energy
                print('\t\t\t\t',ψ[:Width * Height],'= ψ first ambulance destinations','\n','\t\t\t\t',ψ[Width * Height:2*Width * Height],'= ψ 2nd ambulance destinations \t')
                print(ψ[2*Width * Height:3*Width * Height],'= ψ first ambulance Start position',ψ[3*Width * Height:4*Width * Height],'= ψ 2nd ambulance Start position\t', '%3.2f' %E,'= classical Energy', '%3.2f' %datum.energy,'= DIMOD calc of energy n=',n)
print((datetime.now() - tstart).seconds)
print('Finished non-hybrid')


CLASSICAL calculation of energies using dimod and qubo_energy_value() which should equal each other
Location grid is  2 wide and  2 high.


IndexError: list assignment index out of range

    After about 1 hour it completed a 3*2 grid with 4 features, roughly 16m calcs, successfully finding the -524 energy that I expected from the simplicity of the problem. An ambulance in each corner service itself and its two neighbour represents an energy of 4 from the distance covered.

